In [1]:
import pandas as pd
import numpy as np


In [2]:
!git clone https://github.com/jhaaj08/Adobe-India-Hackathon25


Cloning into 'Adobe-India-Hackathon25'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 124 (delta 15), reused 7 (delta 7), pack-reused 87 (from 1)
Receiving objects: 100% (124/124), 19.37 MiB | 34.49 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [14]:
pip install sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 6.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [1]:
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import json

# Load compact model
model = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# Define your task
task = "Prepare a vegetarian buffet-style dinner menu for a corporate gathering, including gluten-free items."
task_embedding = model.encode(task, convert_to_tensor=True)

# ----------------------------
# Example extracted sections (simulate PDF parsing)
sections = [
    {
        "section_title": "Falafel",
        "content": "Ingredients: Chickpeas, onion, garlic, parsley, cumin, coriander. Instructions: Blend, form balls, fry.",
        "document": "Dinner Ideas - Sides_2.pdf",
        "page_number": 7
    },
    {
        "section_title": "Baba Ganoush",
        "content": "Ingredients: Eggplant, tahini, lemon, garlic. Instructions: Roast eggplant, blend ingredients.",
        "document": "Dinner Ideas - Sides_1.pdf",
        "page_number": 4
    },
    {
        "section_title": "Macaroni and Cheese",
        "content": "Ingredients: Elbow macaroni, butter, cheese. Instructions: Boil pasta, make cheese sauce.",
        "document": "Dinner Ideas - Sides_3.pdf",
        "page_number": 8
    },
    {
        "section_title": "Ratatouille",
        "content": "Ingredients: Zucchini, eggplant, bell peppers, tomatoes, herbs. Instructions: Roast or stew vegetables.",
        "document": "Dinner Ideas - Sides_3.pdf",
        "page_number": 8
    },
    {
        "section_title": "Vegetable Lasagna",
        "content": "Ingredients: Lasagna noodles, ricotta, spinach, tomato sauce, mozzarella. Instructions: Layer and bake.",
        "document": "Dinner Ideas - Mains_2.pdf",
        "page_number": 9
    },
    {
        "section_title": "Veggie Sushi Rolls",
        "content": "Ingredients: Sushi rice, cucumber, avocado, carrot, nori. Instructions: Roll and slice.",
        "document": "Lunch Ideas.pdf",
        "page_number": 11
    },
    {
        "section_title": "Escalivada",
        "content": "Ingredients: Eggplants, bell peppers, tomatoes, onion, olive oil. Instructions: Roast, peel, drizzle with oil.",
        "document": "Dinner Ideas - Sides_2.pdf",
        "page_number": 7
    }
]


# ----------------------------
# Rank by semantic similarity
for section in sections:
    section_embedding = model.encode(section["content"], convert_to_tensor=True)
    similarity_score = util.cos_sim(task_embedding, section_embedding).item()
    section["similarity_score"] = similarity_score

# Sort by relevance
ranked_sections = sorted(sections, key=lambda x: -x["similarity_score"])

# ----------------------------
# Create summary from top N (e.g., top 2)
top_n = 8
summary_output = []

for section in ranked_sections[:top_n]:
    summary_output.append({
        "section_title": section["section_title"],
        "document": section["document"],
        "page_number": section["page_number"],
        "similarity_score": round(section["similarity_score"], 3),
        "summary": f"{section['section_title']} is a dish prepared using {section['content'].split('Instructions:')[0].replace('Ingredients:', '').strip()}."
    })

# ----------------------------
# Final structured output
output = {
    "task": task,
    "processing_timestamp": datetime.now().isoformat(),
    "top_sections_ranked": summary_output
}

# ----------------------------
# Print or save
print(json.dumps(output, indent=4))

# Optional: save to file
# with open("ranked_summaries.json", "w") as f:
#     json.dump(output, f, indent=4)
 

KeyboardInterrupt: 

In [ ]:
!git clone https://github.com/jhaaj08/Adobe-India-Hackathon25

In [7]:
from pathlib import Path


[{'level': 'H1', 'text': 'Revision History ', 'page': 2},
 {'level': 'H1', 'text': 'Table of Contents ', 'page': 3},
 {'level': 'H1', 'text': 'Acknowledgements ', 'page': 4},
 {'level': 'H1',
  'text': '1. Introduction to the Foundation Level Extensions ',
  'page': 5},
 {'level': 'H1',
  'text': '2. Introduction to Foundation Level Agile Tester Extension ',
  'page': 6},
 {'level': 'H2', 'text': '2.1 Intended Audience ', 'page': 6},
 {'level': 'H2', 'text': '2.2 Career Paths for Testers ', 'page': 6},
 {'level': 'H2', 'text': '2.3 Learning Objectives ', 'page': 6},
 {'level': 'H2', 'text': '2.4 Entry Requirements ', 'page': 7},
 {'level': 'H2', 'text': '2.5 Structure and Course Duration ', 'page': 7},
 {'level': 'H2', 'text': '2.6 Keeping It Current ', 'page': 8},
 {'level': 'H1',
  'text': '3. Overview of the Foundation Level Extension – Agile TesterSyllabus ',
  'page': 9},
 {'level': 'H2', 'text': '3.1 Business Outcomes ', 'page': 9},
 {'level': 'H2', 'text': '3.2 Content ', 'page'

In [16]:
pip install PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
outline_data

{'title': 'RFP:Request for Proposal To Present a Proposal for Developing the Business Plan for the Ontario Digital Library  ',
 'outline': [{'level': 'H1', 'text': 'Ontario’s Digital Library ', 'page': 1},
  {'level': 'H1',
   'text': 'A Critical Component for Implementing Ontario’s Road Map to Prosperity Strategy ',
   'page': 1},
  {'level': 'H2', 'text': 'Summary ', 'page': 1},
  {'level': 'H3', 'text': 'Timeline: ', 'page': 1},
  {'level': 'H2', 'text': 'Background ', 'page': 2},
  {'level': 'H3', 'text': 'Equitable access for all Ontarians: ', 'page': 3},
  {'level': 'H3',
   'text': 'Shared decision-making and accountability: ',
   'page': 3},
  {'level': 'H3', 'text': 'Shared governance structure: ', 'page': 3},
  {'level': 'H3', 'text': 'Shared funding: ', 'page': 3},
  {'level': 'H3', 'text': 'Local points of entry: ', 'page': 4},
  {'level': 'H3', 'text': 'Access: ', 'page': 4},
  {'level': 'H3', 'text': 'Guidance and Advice: ', 'page': 4},
  {'level': 'H3', 'text': 'Training

In [21]:
from sentence_transformers import SentenceTransformer, util
from PyPDF2 import PdfReader
from datetime import datetime
import json
import re

# ----------------------------
# Load compact model
model = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# Define your task
task = "for citizens what are the guidelines."
task_embedding = model.encode(task, convert_to_tensor=True)

# ----------------------------
# Load the PDF and extract text
pdf_path = "/kaggle/working/Adobe-India-Hackathon25/Challenge_1a/sample_dataset/pdfs/file03.pdf"
reader = PdfReader(pdf_path)

all_text = []
for page in reader.pages:
    all_text.append(page.extract_text())
pdf_text = "\n".join(all_text)

# ----------------------------
# Provided outline
json_path = Path("/kaggle/working/Adobe-India-Hackathon25/Challenge_1a/sample_dataset/outputs/file03.json")  # or full path like "/home/user/pdf_outline.json"

# Load the JSON file
with open(json_path, "r", encoding="utf-8") as f:
    outline_data = json.load(f)
outline = outline_data["outline"]
# Normalize heading text
def normalize(text):
    return re.sub(r'\s+', ' ', text).strip().lower()

# Extract section content from PDF using outline
def extract_sections(outline, full_text):
    sections = []
    headings = [normalize(h["text"]) for h in outline]
    pattern = '|'.join(re.escape(h) for h in headings)
    
    # Split by headings
    matches = list(re.finditer(pattern, normalize(full_text)))
    for idx, match in enumerate(matches):
        start = match.start()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(full_text)
        title = match.group(0)
        section_text = full_text[start:end].strip()
        sections.append({
            "section_title": title.title(),
            "content": section_text
        })
    return sections

extracted_sections = extract_sections(outline, pdf_text)

# ----------------------------
# Rank by semantic similarity
for section in extracted_sections:
    section_embedding = model.encode(section["content"], convert_to_tensor=True)
    similarity_score = util.cos_sim(task_embedding, section_embedding).item()
    section["similarity_score"] = similarity_score

# Sort by relevance
ranked_sections = sorted(extracted_sections, key=lambda x: -x["similarity_score"])

# ----------------------------
# Create summary from top N
top_n = 5
summary_output = []

for section in ranked_sections[:top_n]:
    content_snippet = section['content'][:300].replace('\n', ' ') + '...'
    summary_output.append({
        "section_title": section["section_title"],
        "similarity_score": round(section["similarity_score"], 3),
        "summary": content_snippet
    })

# ----------------------------
# Final structured output
output = {
    "pdf_title": pdf_path.split("/")[-1],
    "task": task,
    "processing_timestamp": datetime.now().isoformat(),
    "top_sections_ranked": summary_output
}

# Print JSON output
print(json.dumps(output, indent=4))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{
    "pdf_title": "file03.pdf",
    "task": "for citizens what are the guidelines.",
    "processing_timestamp": "2025-07-28T14:03:58.053356",
    "top_sections_ranked": [
        {
            "section_title": "Equitable Access For All Ontarians:",
            "similarity_score": 0.416,
            "summary": "common mandates of local libraries.  The principles which will define and guide the ODL are:  Equitable access for all Ontarians:  We will bring consistent, high-quality electronic resources and services to 10 million  Ontario citizens  We will eliminat..."
        },
        {
            "section_title": "Training:",
            "similarity_score": 0.395,
            "summary": "arning to support citizens, in real-time, as they try..."
        },
        {
            "section_title": "5. Term",
            "similarity_score": 0.375,
            "summary": "ips;  \u2022 Knowledgeable about digital libraries and resources;  \u2022 Knowledgeable about government structures;  \u